In [1]:
import os
import json
from text_parser.text_parser import parse,is_valid
from diagram_parser.detection.test import detect_image
import cv2
from tqdm import tqdm
from multiprocess import Pool

In [2]:
OUTPUT_FILE_TEXT = 'output/text/text_logic_forms.json'
OUTPUT_FILE_DIAGRAM = 'output/diagram/diagram_logic_forms.json'

In [3]:
with open('input/data.json', 'r') as f:
    data = json.load(f)

In [4]:
input_text = data['compact_text']
input_text

'Find x'

In [5]:
pid = 'input_diagram'
logic_forms, output_text, reduced_text = parse(input_text)
logic_valid = is_valid(logic_forms)
if logic_valid and len(reduced_text) < 4:
    success = True
    print("Parse Successfully")
else:
    success = False
    print("Parse Failed")


Parse Successfully


In [6]:
output_data = {}
output_data[pid] = {}
output_data[pid]["text_logic_forms"] = logic_forms
output_data[pid]["output_text"] = output_text
output_data[pid]["success"] = success

In [7]:
print("Saving results to {}".format(OUTPUT_FILE_TEXT))
os.makedirs(os.path.dirname(OUTPUT_FILE_TEXT), exist_ok=True)
with open(OUTPUT_FILE_TEXT, 'w') as f:
    json.dump(output_data, f, indent = 2, separators=(',', ': '))

Saving results to output/text/text_logic_forms.json


In [8]:
import os
import glob

# Đường dẫn tới hai thư mục cần xóa tất cả các tệp tin
box_results_dir = "output/diagram/box_results"
ocr_results_dir = "output/diagram/ocr_results"

# Hàm để xóa tất cả các tệp tin trong một thư mục
def delete_files_in_directory(directory):
    files = glob.glob(os.path.join(directory, '*'))
    for f in files:
        try:
            os.remove(f)
            print(f"Deleted file: {f}")
        except OSError as e:
            print(f"Error deleting file {f}: {e}")

# Xóa tất cả các tệp tin trong thư mục box_results
print(f"Deleting all files in directory: {box_results_dir}")
delete_files_in_directory(box_results_dir)

# Xóa tất cả các tệp tin trong thư mục ocr_results
print(f"Deleting all files in directory: {ocr_results_dir}")
delete_files_in_directory(ocr_results_dir)

Deleting all files in directory: output/diagram/box_results
Deleted file: output/diagram/box_results\input_diagram.txt
Deleted file: output/diagram/box_results\input_diagram_modified.jpg
Deleting all files in directory: output/diagram/ocr_results
Error deleting file output/diagram/ocr_results\input_diagram: [WinError 5] Access is denied: 'output/diagram/ocr_results\\input_diagram'


In [9]:
input_image = "input"
detect_model = "diagram_parser\detection\models\csv_retinanet_100.pt"
class_list = "diagram_parser\detection\classes.txt"
output_path = "output\diagram"

In [10]:
detect_image(input_image, detect_model,class_list,output_path)

100%|██████████| 4/4 [00:01<00:00,  3.02it/s]


In [11]:
from diagram_parser.ocr_tool import mathpix
import json
import os
import time
folder = 'output\diagram\ocr_results'
# folder = 'output\\test'
sub_folders = os.listdir(folder)

for sub_folder in sub_folders:
    files = os.listdir(os.path.join(folder, sub_folder))
    files = list(filter(lambda x: x.endswith('jpg'), files))
    
    for file in files:
        path = os.path.join(folder, sub_folder, file)
        r = mathpix.latex({
            'src': mathpix.image_uri(path),
            'formats': ['latex_simplified']
        })
        res = json.dumps(r, indent=4, sort_keys=True)
        # print(res['latex_simplified'])

        id = os.path.splitext(file)[0]
        print(r)
        with open(os.path.join(folder, sub_folder, id+'.json'), 'w') as f:
            f.write(res)
        print(os.path.join(folder, sub_folder, id+'.json'))
        
        time.sleep(1.5)

{'request_id': '2024_06_13_71653414c41b2d064ca6g', 'version': 'RSK-M131p2i1', 'image_width': 41, 'image_height': 30, 'detection_map': {'contains_chart': 0, 'contains_diagram': 0, 'contains_graph': 0, 'contains_table': 0, 'is_blank': 0, 'is_inverted': 0, 'is_not_math': 0, 'is_printed': 0.9992327690124512}, 'detection_list': ['is_printed'], 'auto_rotate_confidence': 9.392809602104535e-05, 'auto_rotate_degrees': 0, 'position': {'top_left_x': 0, 'top_left_y': 0, 'width': 41, 'height': 30}, 'latex_confidence': 1, 'latex_confidence_rate': 1, 'latex_simplified': '60 ^ { \\circ }'}
output\diagram\ocr_results\input_diagram\0.json
{'request_id': '2024_06_13_fc18cf48041b7da0f0e6g', 'version': 'RSK-M131p2i1', 'image_width': 36, 'image_height': 28, 'detection_map': {'contains_chart': 0, 'contains_diagram': 0, 'contains_graph': 0, 'contains_table': 0, 'is_blank': 0, 'is_inverted': 0, 'is_not_math': 0, 'is_printed': 0.9391365647315979}, 'detection_list': ['is_printed'], 'auto_rotate_confidence': 0.00

In [12]:
from diagram_parser.parser import diagram_parser

In [13]:
data_path = "input/data.json"
ocr_path = "output/diagram/ocr_results"
box_path = "output/diagram/box_results"
output_path = OUTPUT_FILE_DIAGRAM

In [14]:
detection_id = ['input_diagram']
box_id = detection_id[0]
data_id = box_id
ocr_results, sign_results, size_results = diagram_parser.load_symbol(detection_id, ocr_path, box_path)
para_lst = []
json_file = data_path
with open(json_file, 'r') as f:
    data = json.load(f)
diagram_path = ('input\diagram\input_diagram.png')
img = cv2.imread(diagram_path)
factor = (1, 1)
if size_results[box_id] is not None:
    factor = (img.shape[1] / size_results[box_id][1], img.shape[0] / size_results[box_id][0])
para_lst.append((data_id, box_id, ocr_results[box_id], sign_results[box_id], data, img, factor))


In [15]:
para_lst

[('input_diagram',
  'input_diagram',
  [(51, 66, 87, 94, '30^{\\circ}'),
   (132, 65, 153, 90, 'X'),
   (156, 159, 197, 189, '60^{\\circ}'),
   (30, 124, 52, 152, 'y'),
   (150, 210, 182, 242, '21')],
  [(82, 191, 119, 227, 'perpendicular')],
  {'compact_text': 'Find x'},
  array([[[255, 255, 255],
          [255, 255, 255],
          [255, 255, 255],
          ...,
          [255, 255, 255],
          [255, 255, 255],
          [255, 255, 255]],
  
         [[255, 255, 255],
          [255, 255, 255],
          [255, 255, 255],
          ...,
          [255, 255, 255],
          [255, 255, 255],
          [255, 255, 255]],
  
         [[255, 255, 255],
          [255, 255, 255],
          [255, 255, 255],
          ...,
          [255, 255, 255],
          [255, 255, 255],
          [255, 255, 255]],
  
         ...,
  
         [[255, 255, 255],
          [255, 255, 255],
          [255, 255, 255],
          ...,
          [255, 255, 255],
          [255, 255, 255],
          [255, 

In [16]:
solve_list = []
with tqdm(total=len(para_lst), ncols=80) as t:
    with Pool(10) as p:
        for answer in p.imap_unordered(diagram_parser.multithread_solve, para_lst):
            solve_list.append(answer)
            t.update()

100%|█████████████████████████████████████████████| 1/1 [00:05<00:00,  5.38s/it]


In [17]:
solve_list = sorted(solve_list, key=lambda x: x['id'])
final = {}
for entry in solve_list:
    id = entry["id"]
    del entry["id"]
    final[id] = entry
with open(output_path, "w") as f:
    json.dump(final, f, indent = 2)

In [18]:
import os
import json
from text_parser.text_parser import parse,is_valid
from diagram_parser.detection.test import detect_image
import cv2
from tqdm import tqdm
from multiprocess import Pool

In [19]:
from theorem_predict.tools.demo_generate_random_seqs import seq_gen_demo

In [20]:
OUTPUT_FILE_TEXT = 'output/text/text_logic_forms.json'
OUTPUT_FILE_DIAGRAM = 'output/diagram/diagram_logic_forms.json'

In [21]:
seq_gen_demo()

input_diagram
{'id': 'input_diagram', 'num_seqs': 1, 'seq': [17, 10, 11, 14, 15, 8, 3, 15, 14, 5, 10, 14, 17, 10, 6, 12, 15, 16, 4, 11, 7, 13, 1, 6, 15, 13, 2, 4, 17, 4, 13, 6, 10, 6, 11, 6, 14, 11, 16, 17, 1, 8, 8, 3, 9, 4, 17, 8, 6, 12, 2, 8, 14, 6, 3, 1, 13, 9, 4, 5, 5, 3, 16, 7, 7, 5, 8, 12, 12, 3, 13, 14, 11, 10, 9, 1, 12, 7, 12, 5, 10, 2, 9, 8, 8, 1, 2, 15, 12, 13, 15, 6, 5, 5, 2, 4, 15, 2, 6, 14]}
Saving generated theorem sequences to output/demo/pred_seqs_test_l100_random.json
input_diagram
{'id': 'input_diagram', 'num_seqs': 1, 'seq': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]}
Saving generated theorem sequences to output/demo/pred_seqs_test_l100_fixed.json


In [2]:
from theorem_predict.demo_eval_transformer import bart_demo
bart_demo()

c:\DUT\Nam4\Ky8\PBL\InterGPS\venvgps\lib\site-packages\transformers\models\bart\modeling_bart.py:597: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [3]:
## 给预测的序列加随机序号

import json
import random
from numpy.random import seed

seed(1234)
random.seed(1234)

# input_file = '../results/test/pred_seq_result_bart_epoch19_seq5.json'
# output_file = '../results/test/pred_seq_result_bart_epoch19_seq5_add_random.json'
input_file = 'output\demo\pred_seqs_test_bart_best.json'
output_file = 'output\demo\pred_seqs_test_bart_best_random.json'
input = json.load(open(input_file))

output = {}
for pid, data in input.items():
    new_seq = data['seq'][0] + random.sample(list(range(1,18))*10, 100)
    output[pid] = {'id':pid, 'num_seqs':1, 'seq':new_seq}
    
## Save result.json file
with open(output_file, 'w') as f:
    json.dump(output, f, indent=2, separators=(',', ': '))

In [4]:
!python symbolic_solver/demo_test.py --data_path input/data.json --label final_new --strategy final --text_logic_form_path output/text/text_logic_forms.json --diagram_logic_form_path output/diagram/diagram_logic_forms.json --predict_path output/demo/pred_seqs_test_bart_best.json

                   label = final_new
                strategy = final
               data_path = input/data.json
    text_logic_form_path = output/text/text_logic_forms.json
 diagram_logic_form_path = output/diagram/diagram_logic_forms.json
           use_annotated = False
            predict_path = output/demo/pred_seqs_test_bart_best.json
             start_index = 2401
               end_index = 3001
              time_limit = 150
             num_threads = 20
               low_first = True
              step_limit = 100
              debug_mode = False
            enable_round = False
          enable_predict = True

Finished, calculating solved problems...
num_solved: 1 (100.00%)
Average steps for solved problems: 17.00
Done.
Saving logging file to: logs/log-input_diagram.log
Saving result file to: pred_results/logic_input_diagram.json



100%|█████████████████████████████████████████████| 1/1 [00:07<00:00,  7.34s/it]


In [5]:
# import subprocess
# import sys
# venv_python = sys.executable
# command = [
#     venv_python, 'symbolic_solver/demo_test.py',
#     '--data_path', 'input/data.json',
#     '--label', 'final_new',
#     '--strategy', 'final',
#     '--text_logic_form_path', 'output/text/text_logic_forms.json',
#     '--diagram_logic_form_path', 'output/diagram/diagram_logic_forms.json',
#     '--predict_path', 'output/demo/pred_seqs_test_bart_best.json'
# ]
# # Thực hiện lệnh
# result = subprocess.run(command, capture_output=True, text=True)

# # In ra kết quả
# print("stdout:", result.stdout)
# print("stderr:", result.stderr)
# print("Return code:", result.returncode)

In [6]:
# import json

# # Đọc dữ liệu từ file JSON thứ nhất
# with open('output/logic_form_edited.json', 'r') as file:
#     edited_data = json.load(file)
# # Đọc dữ liệu từ file JSON thứ hai và ba
# with open('output/text/text_logic_forms.json', 'r') as file:
#     text = json.load(file)

# with open('output/diagram/diagram_logic_forms.json', 'r') as file:
#     diagram = json.load(file)

In [7]:
# # Hàm để thay thế các giá trị của trường trong đối tượng JSON
# def replace_values(source, target):
#     for key, value in source.items():
#         if key in target['input_diagram']:
#             # print(key)
#             if isinstance(value, dict) and isinstance(target['input_diagram'][key], dict):
#                 target['input_diagram'][key] = value 
#             else:
#                 target['input_diagram'][key] = value

# # Thay thế các giá trị trong text_data nếu tồn tại trong edit_data
# replace_values(edited_data, text)

# # Thay thế các giá trị trong diagram_data nếu tồn tại trong edit_data
# replace_values(edited_data, diagram)

# # Ghi dữ liệu đã thay thế vào file text_updated.json và diagram_updated.json
# with open('output/text/text_logic_forms.json', 'w', encoding='utf-8') as file:
#     json.dump(text, file, ensure_ascii=False, indent=4)

# with open('output/diagram/diagram_logic_forms.json', 'w', encoding='utf-8') as file:
#     json.dump(diagram, file, ensure_ascii=False, indent=4)